In [0]:
from google.colab import drive

In [3]:
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

In [0]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Hashing")

In [0]:
import re
import jieba
import pandas as pd

In [0]:
train_data_path = "./gen_data/AutoMaster_TrainSet.csv"
test_data_path = "./gen_data/AutoMaster_TestSet.csv"

In [0]:
def load_dataset(train_data_path: str, test_data_path: str):
  
  train_data = pd.read_csv(train_data_path)
  test_data = pd.read_csv(test_data_path)

  return train_data, test_data

In [0]:
train_df, test_df = load_dataset(train_data_path, test_data_path)

In [10]:
print("train data size: {}, test data size: {}".format(len(train_df), len(test_df)))

train data size: 82943, test data size: 20000


In [11]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


In [0]:
## 1. 空值处理

In [0]:
train_df.dropna(subset=['Question', 'Dialogue', 'Report'], how='any', inplace=True)
test_df.dropna(subset=['Question', 'Dialogue'], how='any', inplace=True)

In [14]:
print("after dropna train data size: {}, test data size: {}".format(len(train_df), len(test_df)))

after dropna train data size: 82871, test data size: 20000


In [0]:
## 2. 去除无用字符

In [0]:
def clean_sentence(sentence):
  if isinstance(sentence, str):
    return re.sub(
        r'[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】“”！，。？、~@#￥%……&*（）]+|车主说|技师说|语音|图片|你好|您好',
        '', sentence)
  else:
    return ""

In [17]:
sentence='2012款奔驰c180怎么样，维修保养，动力，值得拥有吗'
print('orign sentence :{} \n'.format(sentence))
print('clean sentence: {} \n'.format(clean_sentence(sentence)))

orign sentence :2012款奔驰c180怎么样，维修保养，动力，值得拥有吗 

clean sentence: 2012款奔驰c180怎么样维修保养动力值得拥有吗 



In [0]:
## 3. 切词

In [19]:
sentence='2010款的宝马X1，2011年出厂，2.0排量'
print(list(jieba.cut(sentence)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.879 seconds.
Prefix dict has been built successfully.


['2010', '款', '的', '宝马', 'X1', '，', '2011', '年', '出厂', '，', '2.0', '排量']


In [0]:
# 自定义分词字典
user_dict='./common_data/user_dict.txt'

In [21]:
# 载入自定义词典
jieba.load_userdict(user_dict)
print(list(jieba.cut(sentence)))

['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量']


In [0]:
## 4. 去除停用词

In [0]:
# 停用词路径
stopwords_path = "./common_data/stopwords/哈工大停用词表.txt"

In [0]:
def load_stop_words(stopwords_path):
  with open(stopwords_path, "r") as f:
    stop_words = [word.strip() for word in f.readlines()] 
    stop_words_dict = dict(zip(stop_words, range(len(stop_words))))
  
  return stop_words, stop_words_dict

In [25]:
%%time
stop_words, stop_words_dict = load_stop_words(stopwords_path)
print('stop words size {}'.format(len(stop_words)))

stop words size 767
CPU times: user 1.92 ms, sys: 0 ns, total: 1.92 ms
Wall time: 7.63 s


In [0]:
# 过滤停用词
def filter_stopwords(words, stop_words=None, stop_words_dict=None):
  if stop_words:
    return [word for word in words if word not in stop_words]
  if stop_words_dict:
    return [word for word in words if word not in stop_words_dict]

In [27]:
%%time
print('orign sentence :{} \n'.format(sentence))
words = list(jieba.cut(sentence))
print('words: {} \n'.format(words))
print('filter stop word : {} '.format(filter_stopwords(words, stop_words)))

orign sentence :2010款的宝马X1，2011年出厂，2.0排量 

words: ['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量'] 

filter stop word : ['2010', '款', '宝马X1', '2011', '年', '出厂', '2.0', '排量'] 
CPU times: user 495 µs, sys: 0 ns, total: 495 µs
Wall time: 477 µs


In [28]:
%%time
print('orign sentence :{} \n'.format(sentence))
words = list(jieba.cut(sentence))
print('words: {} \n'.format(words))
print('filter stop word : {} '.format(filter_stopwords(words, stop_words_dict)))

orign sentence :2010款的宝马X1，2011年出厂，2.0排量 

words: ['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量'] 

filter stop word : ['2010', '款', '宝马X1', '2011', '年', '出厂', '2.0', '排量'] 
CPU times: user 439 µs, sys: 0 ns, total: 439 µs
Wall time: 385 µs


In [0]:
## 5. 预处理流程合并

In [0]:
def sentence_proc(sentence):

  sentence = clean_sentence(sentence)
  stop_words, stop_words_dict = load_stop_words(stopwords_path)
  words = jieba.cut(sentence)
  words = filter_stopwords(words, stop_words_dict)

  return " ".join(words)

In [40]:
sentence

'2010款的宝马X1，2011年出厂，2.0排量'

In [41]:
sentence_proc(sentence)

'2010 款 宝马X1 2011 年 出厂 20 排量'

In [42]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


In [0]:
## 6. 批量预处理

In [0]:
def data_frame_proc(df):
  # 批量预处理 训练集和测试集
  for col_name in ['Brand', 'Model', 'Question', 'Dialogue']:
      df[col_name] = df[col_name].apply(sentence_proc)

  if 'Report' in df.columns:
      # 训练集 Report 预处理
      df['Report'] = df['Report'].apply(sentence_proc)
  return df

In [45]:
%%time
train_df = data_frame_proc(train_df)
test_df = data_frame_proc(test_df)

CPU times: user 6min 40s, sys: 1min 6s, total: 7min 46s
Wall time: 12min 41s


In [46]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1 进口,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先 挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场价 钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


In [0]:
## 7. 合并训练测试数据

In [49]:
train_df["merged"] = train_df[["Question", "Dialogue", "Report"]].apply(lambda x: " ".join(x), axis=1)
test_df['merged'] = test_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
merged_df = pd.concat([train_df[['merged']], test_df[['merged']]], axis=0)
print('train data size {},test data size {},merged_df data size {}'.format(len(train_df), len(test_df),len(merged_df)))

train data size 82871,test data size 20000,merged_df data size 102871


In [52]:
merged_df.head()

,merged
0,方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重...
1,奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 有发 一下 发动...
2,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...
3,30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 ...
4,2012 款 奔驰 c180 维修保养 动力 值得 拥有 家庭 用车 入手 维修保养 费用 ...


In [0]:
# 保存数据
train_df = train_df.drop(['merged'], axis=1)
test_df = test_df.drop(['merged'], axis=1)
train_df.to_csv("./gen_data/train_seg_data.csv", index=None, header=True)
test_df.to_csv('./gen_data/test_seg_data.csv', index=None, header=True)
merged_df.to_csv("./gen_data/merged_train_test_seg_data.csv", index=None, header=False)

In [0]:
## 8. 训练词向量

In [0]:
from gensim.models.word2vec import LineSentence, Word2Vec

In [0]:
merged_seg_path = "./gen_data/merged_train_test_seg_data.csv"

In [59]:
wv_model = Word2Vec(LineSentence(merged_seg_path), size=300, negative=5, workers=8, iter=3, window=3, min_count=5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 生成词表

In [62]:
vocab = {word: index for index, word in enumerate(wv_model.wv.index2word)}
reverse_vocab = {index: word for index, word in enumerate(wv_model.wv.index2word)}
len(vocab)

32905

In [0]:
## 获取词向量矩阵

In [64]:
embedding_matrix = wv_model.wv.vectors
embedding_matrix.shape

(32905, 300)

In [0]:
## 9. 构建训练数据

* 可以把Question,Dialogue当做一句 长文本处理, 合并构建成X
* Report作为需要预测的标签,构建Y

In [0]:
train_df['X'] = train_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
test_df['X'] = test_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)

In [67]:
train_df['X'].head()

0    方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重...
1    奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 有发 一下 发动...
2    2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...
3    30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 ...
4    2012 款 奔驰 c180 维修保养 动力 值得 拥有 家庭 用车 入手 维修保养 费用 ...
Name: X, dtype: object

In [0]:
## 获取样本输入max_len值

In [0]:
def get_max_len(seg_sentence):
  max_lens = seg_sentence.apply(lambda x: x.count(" "))
  return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [71]:
import numpy as np
train_y_max_len = get_max_len(train_df['X'])
test_y_max_len = get_max_len(test_df['X'])

x_max_len = max(train_y_max_len, test_y_max_len)

train_y_max_len = get_max_len(train_df['Report'])

print("train_x max lens: {0}, train_y max lens: {1}".format(x_max_len, train_y_max_len))

train_x max lens: 258, train_y max lens: 31


In [0]:
## 开始填充

In [0]:
def pad_process(sentence, max_len, vocab):

  words = sentence.strip().split(" ")
  words = words[: max_len]
  sentence = [word if word in vocab else "<UNK>" for word in words]
  sentence = ["<START>"] + sentence + ["<STOP>"]
  sentence = sentence + ["<PAD>"] * (max_len - len(words))

  return " ".join(sentence)

In [0]:
# 训练集X处理
train_df['X'] = train_df['X'].apply(lambda x: pad_process(x, x_max_len, vocab))
# 训练集Y处理
train_df['Y'] = train_df['Report'].apply(lambda x: pad_process(x, train_y_max_len, vocab))
# 测试集X处理
test_df['X'] = test_df['X'].apply(lambda x: pad_process(x, x_max_len, vocab))

In [76]:
train_df["X"].head()

0    <START> 方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向...
1    <START> 奔驰 <UNK> 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 ...
2    <START> 2010 款 宝马X1 2011 年 出厂 20 排量 通用 <UNK> 变...
3    <START> 30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 ...
4    <START> 2012 款 奔驰 c180 维修保养 动力 值得 拥有 家庭 用车 入手 ...
Name: X, dtype: object

In [0]:
## 保存填充好的数据

In [0]:
train_x_pad_path = "./gen_data/train_X_pad_data.csv"
train_y_pad_path = "./gen_data/train_Y_pad_data.csv"
test_x_pad_path = "./gen_data/test_X_pad_data.csv"

train_df['X'].to_csv(train_x_pad_path, index=None, header=False)
train_df['Y'].to_csv(train_y_pad_path, index=None, header=False)
test_df['X'].to_csv(test_x_pad_path, index=None, header=False)

In [0]:
## 新加的符号不在词表 和 词向量矩阵中,怎么办?
## 重新训练词表


In [79]:
print('start retrain w2v model')
wv_model.build_vocab(LineSentence(train_x_pad_path), update=True)
wv_model.train(LineSentence(train_x_pad_path), epochs=3, total_examples=wv_model.corpus_count)
print('1/3')
wv_model.build_vocab(LineSentence(train_y_pad_path), update=True)
wv_model.train(LineSentence(train_y_pad_path), epochs=3, total_examples=wv_model.corpus_count)
print('2/3')
wv_model.build_vocab(LineSentence(test_x_pad_path), update=True)
wv_model.train(LineSentence(test_x_pad_path), epochs=3, total_examples=wv_model.corpus_count)

start retrain w2v model


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


1/3
2/3


(5214916, 15600000)

In [80]:
# 保存词向量模型
save_wv_model_path = './gen_data/word2vec.model'
wv_model.save(save_wv_model_path)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [81]:
#更新vocab
vocab = {word: index for index, word in enumerate(wv_model.wv.index2word)}
reverse_vocab = {index: word for index, word in enumerate(wv_model.wv.index2word)}
# 更新词向量矩阵
embedding_matrix = wv_model.wv.vectors
embedding_matrix.shape

(32909, 300)

In [0]:
def save_dict(save_path, vocab):
  with open(save_path, "w", encoding="utf-8") as f:
    for k, v in vocab.items():
      f.write("{}\t{}\n".format(k, v))

In [0]:
# 保存vocab
save_dict("./gen_data/vocab.txt", vocab)
save_dict("./gen_data/reverse_vocab.txt", reverse_vocab)

In [0]:
## 训练测试数据，数值转换

In [0]:
# 遇到未知词就填充unk的索引
unk_index = vocab['<UNK>']
def transform_data(sentence,vocab):
  # 字符串切分成词
  words = sentence.split(' ')
  # 按照vocab的index进行转换
  ids = [vocab[word] if word in vocab else unk_index for word in words]
  return ids

In [0]:
# 将词转换成索引  [<START> 方向机 重 ...] -> [32800, 403, 986, 246, 231
train_ids_x = train_df["X"].apply(lambda x:transform_data(x,vocab))
train_ids_y = train_df["Y"].apply(lambda x:transform_data(x,vocab))
test_ids_x = test_df["X"].apply(lambda x:transform_data(x,vocab))

In [0]:
train_X = np.array(train_ids_x.tolist())
train_Y = np.array(train_ids_y.tolist())
test_X = np.array(test_ids_x.tolist())

In [94]:
len(train_df["X"])

82871

In [95]:
print("shape of train_X:", train_X.shape)
print("shape of train_Y:", train_Y.shape)
print("shape of test_X:", test_X.shape)

shape of train_X: (82871, 260)
shape of train_Y: (82871, 33)
shape of test_X: (20000, 260)


In [0]:
# 保存最终数据
np.savetxt("./gen_data/train_X", train_X, fmt='%0.8f')
np.savetxt("./gen_data/train_Y", train_Y, fmt='%0.8f')
np.savetxt("./gen_data/test_X", test_X, fmt='%0.8f')

In [0]:
## 10. 简易模型搭建

In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [3]:
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
tf.config.experimental.list_physical_devices(device_type='CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
print('GPU:', tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU: True


In [5]:
import timeit


with tf.device('/cpu:0'):
	cpu_a = tf.random.normal([10000, 1000])
	cpu_b = tf.random.normal([1000, 2000])
	print(cpu_a.device, cpu_b.device)

with tf.device('/gpu:0'):
	gpu_a = tf.random.normal([10000, 1000])
	gpu_b = tf.random.normal([1000, 2000])
	print(gpu_a.device, gpu_b.device)

def cpu_run():
	with tf.device('/cpu:0'):
		c = tf.matmul(cpu_a, cpu_b)
	return c 

def gpu_run():
	with tf.device('/gpu:0'):
		c = tf.matmul(gpu_a, gpu_b)
	return c 


# warm up
cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('warmup:', cpu_time, gpu_time)


cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('run time:', cpu_time, gpu_time)


/job:localhost/replica:0/task:0/device:CPU:0 /job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0 /job:localhost/replica:0/task:0/device:GPU:0
warmup: 3.5751486059999706 1.4547538699999905
run time: 3.468911317999982 0.0005316620000712646


In [0]:
from tensorflow.python.client import device_lib

In [7]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15847827231927116538, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16192029989427960719
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16302903986768164798
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7304675328
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8998069186262333422
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]